In [1]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
MONGO_URI = os.getenv("MONGO_URI")
ALPHAVANTAGE_API_KEY = os.getenv("ALPHAVANTAGE_API_KEY")

In [2]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient(MONGO_URI)
db = client["robo_advisor"]  # Use the "robo_advisor" database
collection = db["historical_prices"]  # Use the "market_data" collection

In [3]:
# Fetch distinct tickers from MongoDB
tickers = db['historical_prices'].distinct("Asset")
print(f"Fetched {len(tickers)} tickers from MongoDB")

Fetched 71 tickers from MongoDB


In [4]:
# Initialize a list to store general information
data = []

In [5]:
import yfinance as yf
import pandas as pd

# Fetch sector and industry information for each ticker
for ticker in tickers:
    try:
        # Download ticker info
        stock = yf.Ticker(ticker)
        info = stock.info
        
        # Append the sector and industry details
        data.append({
            "Ticker": ticker,
            "Short Name": info.get("shortName", "N/A"),
            "Long Name": info.get("longName", "N/A"),
            "Asset_Type": info.get("quoteType", "N/A"), 
            "Sector": info.get("sector", "N/A"),
            "Industry": info.get("industry", "N/A"),
            "Exchange": info.get("exchange", "N/A"),
            "Currency": info.get("currency", "N/A"),
            "Description": info.get("longBusinessSummary", "N/A")
        })
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        data.append({"Ticker": ticker, "Sector": "Error", "Industry": "Error", "Asset_Type": "Error"})

# Convert the data into a Pandas DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

   Ticker                       Short Name  \
0    AAPL                       Apple Inc.   
1     AGG  iShares Core U.S. Aggregate Bon   
2     AMD     Advanced Micro Devices, Inc.   
3    AMGN                       Amgen Inc.   
4    AMZN                 Amazon.com, Inc.   
..    ...                              ...   
66    XLF  SPDR Select Sector Fund - Finan   
67    XLK  SPDR Select Sector Fund - Techn   
68    XLV  SPDR Select Sector Fund - Healt   
69    XLY  SPDR Select Sector Fund - Consu   
70     ZG               Zillow Group, Inc.   

                                            Long Name Asset_Type  \
0                                          Apple Inc.     EQUITY   
1                iShares Core U.S. Aggregate Bond ETF        ETF   
2                        Advanced Micro Devices, Inc.     EQUITY   
3                                          Amgen Inc.     EQUITY   
4                                    Amazon.com, Inc.     EQUITY   
..                                     

In [6]:
# Find N/A rows
df.isin(['N/A']).sum()

Ticker          0
Short Name      0
Long Name       0
Asset_Type      0
Sector         21
Industry       21
Exchange        0
Currency        0
Description     0
dtype: int64

In [22]:
# Find rows where any column contains "N/A"
other_columns = df.drop(columns=['Ticker'])
na_rows = other_columns.eq('N/A').any(axis=1)
tickers_with_na = df.loc[na_rows, ['Ticker', 'Asset_Type']]

# Display rows with "N/A"
print(tickers_with_na)


   Ticker Asset_Type
1     AGG        ETF
9     BND        ETF
19    DIA        ETF
21    EEM        ETF
22    EFA        ETF
25    GLD        ETF
27    HYG        ETF
29    IEF        ETF
32    IWM        ETF
37    LQD        ETF
50    QQQ        ETF
51    SHY        ETF
52    SPY        ETF
55    TLT        ETF
60    VEA        ETF
64    VWO        ETF
65    XLE        ETF
66    XLF        ETF
67    XLK        ETF
68    XLV        ETF
69    XLY        ETF


In [ ]:
# import requests

# # AlphaVantage API key
# api_key = ALPHAVANTAGE_API_KEY

# # Function to fetch metadata for a ticker
# def get_symbol_metadata(symbol):
#     url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={api_key}"
#     response = requests.get(url).json()  # Fetch the response and parse it as JSON
    
#     # Print the raw response for inspection
#     print(f"Response for {symbol}:")
#     print(response)
#     return response

# response = get_symbol_metadata("AGG")

Response for AGG:
{'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}
